In [1]:
import os
import cv2
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt
 
from moviepy.editor import *
%matplotlib inline
 
from sklearn.model_selection import train_test_split
 
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [2]:
# using this parameter makes sure that anyone who re-runs your code will get the exact same outputs.
seed_constant = 27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [15]:
# frame size
Image_height, Image_width = 64, 64

# specify the number of frames of a video that will be fed to the model as one sequence.
sequence_length = 30

# dataset path
Dataset_dir = "UFC50"

# Activities that will predict
class_list =  ["Basketball", "Biking", "WalkingWithDog", "HorseRace", "PlayingGuitar", "PlayingPiano", "Punch", "Skiing", "GolfSwing"]

In [18]:
# this function will extract the required frames from a video after resizing and normalizing them.
def frames_extraction(video_path):
    
    # list to store frames in video
    frames_list = []

    # read the video
    video_reader = cv2.VideoCapture(video_path)
    
    # total numbers of frame in the video
    video_frames_count = int(video_reader(cv2.CAP_PROP_FRAME_COUNT))
    
    # calculate the the interval after which frames will be added to the list
    skip_frames_window = max(int(video_frames_count/sequence_length), 1)
    
    for frame_counter in range(sequence_length):
        
        # set the current frame position of the video
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        
        # reading the frame from the video.
        success, frame = video_reader.read() 
        
        if not success:
            break
 
         # resize the Frame to fixed height and width.
        resized_frame = cv2.resize(frame, (Image_height, Image_width))
        
        #  normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255
        
        # append the normalized frame into the frames list
        frames_list.append(normalized_frame)
    
    video_reader.release()

    return frames_list


In [19]:
# this function will extract the data of the selected classes and create the required dataset.

def create_dataset():
   
    # features:          A list containing the extracted frames of the videos.
    # labels:            A list containing the indexes of the classes associated with the videos.
    # video_files_paths: A list containing the paths of the videos in the disk.

 
    # Declared Empty Lists to store the features, labels and video file path values.
    features = []
    labels = []
    video_files_paths = []
    
    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(class_list):
        
        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of Class: {class_name}')
        
        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(os.path.join(Dataset_dir, class_name))
        
        # Iterate through all the files present in the files list.
        for file_name in files_list:
            
            # Get the complete video path.
            video_file_path = os.path.join(Dataset_dir, class_name, file_name)
 
            # Extract the frames of the video file.
            frames = frames_extraction(video_file_path)
 
            # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
            # So ignore the vides having frames less than the SEQUENCE_LENGTH.
            if len(frames) == sequence_length:
 
                # Append the data to their repective lists.
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)
 
    # Converting the list to numpy arrays
    features = np.asarray(features)
    labels = np.array(labels)  
    
    # Return the frames, class index, and video file path.
    return features, labels, video_files_paths

In [20]:
# Create the dataset.
features, labels, video_files_paths = create_dataset()

NameError: name 'CLASSES_LIST' is not defined